In [1]:
%load_ext autoreload
%autoreload 2

# Background

When building Large Language Model (LLM) applications, we strive to balance between achieving the highest response quality while adhering to a limited cost budget. Closed models like GPT-4 are renowned for their superior quality, but they can become prohibitively expensive, especially when handling a large volume of queries. On the other hand, Open Source Software (OSS) models are more cost-effective but may not deliver the same quality, particularly for complex or domain-specific queries.

A "smart router" addresses this challenge by processing user queries and deciding whether to route them to a closed LLM or an OSS LLM, depending on the query's complexity or domain. Here’s a schematic representation of a smart router:
![Smart Router](assets/router_schema.png)

Given a set of user queries, a smart router enables generating high-quality LLM responses while minimizing the overall cost.

# Approach

In this tutorial, we'll demonstrate how to train a smart router on the Anyscale platform. We make the following design choices:

1. **Model Choices**: We’ll use GPT-4 as an example of a closed LLM and Mixtral-8x7B as the OSS LLM, so our smart router will route between these two models.
2. **Response Quality Rating**: We'll quantify the quality of an LLM response on a scale of 1 to 5 stars, with higher scores indicating better quality. For simplicity, we'll assume that GPT-4 always achieves a 5-star rating, so it serves as a reference for Mixtral-8x7B.
3. **Smart Router Model**: We'll finetune a Llama3-8B model as our smart router and leverage Anyscale's powerful API. Our research (see our [arXiv paper](put link to arxiv paper)) shows that this model offers superior routing performance compared to smaller architectures.

More concretely, the objective of a smart router is to direct simple queries to Mixtral-8x7B, thereby maintaining high overall response quality (e.g., an average score of 4.8/5) while significantly reducing costs (e.g., by 50%).



# Table of Contents

### Steps to Train a Smart Router

1. [**Prepare labeled data**](#generate-labeled-data): We describe in this section how we generate synthetic labeled data to train the smart router model.

2. [**Finetune a router model**](#finetune-router-model)
TODO

1. [**Evaluate router model offline**](#evaluate-router-model-offline)
TODO



# Step 1: Prepare Labeled Data <a id="generate-labeled-data"></a>

Our smart router essentially functions as a binary classifier, deciding whether to route a query to GPT-4 or Mixtral-8x7B based on the query text. Initially, we considered labeled data in the format `(query, routing_label)`, where `routing_label` is 1 if the query should be routed to Mixtral-8x7B and 0 if it should be routed to GPT-4.

However, our early experiments revealed that *binary labels do not provide sufficient signal for training a robust router model*. Therefore, we adopted a different labeling approach using a *1-5 scoring system*, which reflects how well Mixtral-8x7B can effectively respond to the user's query. More specifically:

- **4-5**: Mixtral-8x7B produces a very strong answer, showing deep understanding, creativity, detailed insight, and high relevance.
- **3**: Mixtral-8x7B provides an adequate answer with moderate detail, relevance, and factual accuracy.
- **1-2**: Mixtral-8x7B struggles to produce a strong answer due to the question's difficulty, vagueness, or the model's limitations.

We use labeled samples in the format `(query, score_label)`. The `routing_label` can be derived from the `score_label` by setting a score threshold for quality, i.e. `routing_label = 1 if score_label >= 4 else 0`.

In the following, we will explain how we prepare our labeled dataset in detail.


<!-- - Nectar dataset helps reduce costs by providing a variety of queries with corresponding model responses. We select queries with GPT-4 responses, generate additional responses from Mixtral-8x7B, and then obtain pairwise comparison labels using llm-as-a-judge (reference paper).

- By following these steps, we collect a preference dataset of approximately 120,000 samples, with a total cost of around $700 USD. -->

## 1.1: Query Dataset

We want our smart router to be effective in open-ended chat domains. So, our first step is to collect a set of generic queries from the [Nectar dataset](https://huggingface.co/datasets/berkeley-nest/Nectar). We chose the Nectar dataset for two reasons: it combines queries from many different domains, including open-ended chat, and it has responses from many models, including over 191K responses from GPT-4.

In [5]:
from data_utils import load_and_display_nectar
nectar_df = load_and_display_nectar()

,prompt,answers,turns,num_responses,source,good_natured
0,\n\nHuman: 0.002 = 1000 \n1 = x?\n\nAssistant:,"[{'answer': 'To find the value of x, we can set up a proportion using the given information: 0.002/1000 = 1/x To solve for x, we can cross multiply: 0.002 * x = 1000 * 1 0.002x = 1000 Dividing both sides by 0.002: x = 1000 / 0.002 x = 500,000 Therefore, 1 is equal to 500,000 in this proportion.', 'model': 'gpt-3.5-turbo', 'rank': 1.0}, {'answer': 'To solve for x in this equation, you can use cross-multiplication. Here's how: 0.002 / 1 = 1000 / x Cross-multiplication gives: 0.002 * x = 1000 To solve for x, divide both sides by 0.002: x = 1000 / 0.002 So, x equals 500,000.', 'model': 'gpt-4-0613', 'rank': 2.0}, {'answer': 'This looks like a proportion. To solve for x, you can set up a proportion equation: 0.002 / 1 = 1000 / x Now, cross-multiply: 0.002 \* x = 1 \* 1000 Then, solve for x: 0.002x = 1000 x = 1000 / 0.002 x = 500,000', 'model': 'gpt-3.5-turbo', 'rank': 3.0}, {'answer': 'If 0.002 equals 1000, then 1 would equal 500,000.', 'model': 'gpt-4', 'rank': 4.0}, {'answer': 'I apologize, but the equation you provided is not correct. 0.002 is equal to 0.002, not 1000. 1 is equal to x, not a number. Can you please provide a correct equation or problem for me to solve?', 'model': 'llama-2-7b-chat', 'rank': 5.0}, {'answer': '0.001 = x', 'model': 'gpt-3.5-turbo-instruct', 'rank': 6.0}, {'answer': 'It seems like you are asking for the value of x in the equation x = 1/0.002. To solve this equation, you can divide both sides by 0.002 to get: x = 1/0.002 x = 500 Therefore, x = 500.', 'model': 'mistral-7b-instruct-v0.1', 'rank': 7.0}]",1,7,[sharegpt],True


Number of queries with GPT-4 responses: 191487


## 1.2 Data Preprocessing

We will use a subset of the Nectar data that includes responses from GPT-4, as these will be used to generate scores (as seen below). We will process this data by focusing on single-turn conversations, filtering for good-natured interactions, and cleaning up the prompts and responses to maintain high quality. Additionally, we will sample a small subset from the dataset for the purpose of this tutorial; however, you can skip sampling to work with the full dataset.

In [6]:
from data_utils import preprocess_nectar

nectar_gpt4_df = preprocess_nectar(nectar_df, "gpt-4", "gpt4")

# Sample a small subset from the dataset for the purpose of this tutorial
N_SUBSET = 10
dataset_df = nectar_gpt4_df.sample(N_SUBSET, random_state=42)

## 1.3 Data Labeling

We don't have human labels for scores, so we will use the [LLM-as-a-Judge approach](https://arxiv.org/abs/2306.05685). GPT-4 will act as an evaluator, reviewing the query and Mixtral's response to provide a score from 1-5. As shown in the paper, the most robust way to get labels is by providing a reference answer for comparison. Here, GPT-4's own response serves as the reference, and Mixtral's response is evaluated against it.

There are two main steps in this process:
1. **Generate Mixtral-8x7B responses for all queries**: We will use an online batch-inference method utilizing Ray and Anyscale endpoints.
2. **Generate LLM-as-a-Judge labels**: We will ask GPT-4 to evaluate the Mixtral responses against its own reference answers and provide a score from 1-5.

### Generate Mixtral-8x7B Responses

In [9]:
from dotenv import load_dotenv
import os
from online_inference import generate_mixtral_responses

# store your API key in a .env file in the home directory
load_dotenv("/home/ray/.env")
anyscale_api_key = os.getenv("ANYSCALE_API_KEY")

dataset_df = generate_mixtral_responses(dataset_df, anyscale_api_key)


Starting batch inference on 10 queries...
# queries un-processed: 9, in-progress: 1, ready: 0
# queries un-processed: 8, in-progress: 2, ready: 0
# queries un-processed: 7, in-progress: 3, ready: 0
# queries un-processed: 6, in-progress: 4, ready: 0
# queries un-processed: 5, in-progress: 5, ready: 0
# queries un-processed: 4, in-progress: 6, ready: 0
# queries un-processed: 3, in-progress: 7, ready: 0
# queries un-processed: 2, in-progress: 8, ready: 0
# queries un-processed: 1, in-progress: 9, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-processed: 0, in-progress: 10, ready: 0
# queries un-

,prompt,source,gpt4,mixtral
6062,"Based on the features mentioned, which hotel d...",[evol_instruct],"Based on the features mentioned, Hotel A seems...","Based on the information provided, I would ne..."
113830,Provide step-by-step instructions on how to cr...,[ultrachat],"Sure, here's a simple step-by-step guide on ho...","Sure, I'd be happy to help you make a homemad..."
138869,What are the 10 largest cities in the US by po...,[lmsys-chat-1m],"As of the most recent data available, the 10 l...",Here are the 10 largest cities in the U.S. by...
169249,Write a comparison essay of at least 500 words...,[ultrachat],Title: A Comparative Analysis of Driving a Car...,Title: The Great Debate: Driving a Car vs. Ri...
116934,"Q: You are provided with an ""Event"", ""Intent"" ...",[flan_v2_niv2],PersonX might feel satisfied or content using ...,Person X probably feels comfortable and focus...


Saving results to assets/test_dataset.jsonl.


### Generate LLM-as-a-judge labels

Let's first take a look at an example query we will send to GPT-4 for judgement

In [16]:

import json
from data_utils import format_judge_prompt


with open("assets/judge_template.json") as f:
    judge_template = json.load(f)

example = dataset_df.iloc[4]
print(
    format_judge_prompt(
        judge_template,
        question=example["prompt"],
        answer=example["mixtral"],
        reference_answer=example["gpt4"],
    )
)

[Instruction]
Evaluate the AI assistant's proficiency in answering the user question displayed below. Your evaluation should consider factors such as the helpfulness, relevance, adherence to real-world facts, depth, creativity, and level of detail of the response. You will be given a reference answer which is considered of high quality. Your assessment will have two lines: First line has a rating on a scale of 1 to 5 with a higher rating representing higher response quality. Follow strictly this format: "[[rating]]", for example: "[[3]]". Second line contains a short explanation of your rating.

[Question]
Q: You are provided with an "Event", "Intent" related to PersonX. Guess a reaction/reaction of PersonX about the given event and their intention.
Event:PersonX uses ___ in class. Intent: 1) to use his prefered writing implement
A:

[Reference Answer]
PersonX might feel satisfied or content using their preferred writing implement in class, as it aligns with their intention to utilize 

Now, we apply a similar online batch-inference method to generate our labels.

In [ ]:
from dotenv import load_dotenv
import os
from online_inference import generate_llm_judge_labels

# store your API key in a .env file in the home directory
load_dotenv("/home/ray/.env")
openai_api_key = os.getenv("OPENAI_API_KEY")

dataset_df = generate_llm_judge_labels(dataset_df, openai_api_key)

# Step 2: Finetune a router model <a id="finetune-router-model"></a>

In this section we will explain how you can finetune an LLM as a smart router. While we have described above how to  Note that while our data contains `gpt4_response` and `mixtral_response`, we will only use the pair `(query, label)` in training our model. At the end of the day, the smart router is supposed to rely on the query text only to infer which model to route to. Our approach is simple: we train a 5-way classifier to predict the score given the query. 


In [ ]:
import pandas as pd

dataset_df = pd.read_json(
    "/mnt/user_storage/templates_data/labeled_full_dataset.jsonl",
    lines=True,
    orient="records",
)

In [ ]:
from data_utils import visualize_label_distribution

# visualize the label distribution
visualize_label_distribution(dataset_df, key="label")

Explain why we higher % for 4-5 scores (Mixtral is competitive with GPT-4 06-2023 version)

Let us assume that if the score >= 4 then we will route to the OSS model (the response quality is good enough), otherwise, we will route the closed model. Under this assumption, the data distribution looks like:

In [ ]:
dataset_df["routing_label"] = dataset_df["label"].apply(lambda x: 1 if x >= 4 else 0)
visualize_label_distribution(dataset_df, key="routing_label")

In [ ]:
from data_utils import split_dataset, balance_dataset

# split data to train/validation sets
train_df, validation_df = split_dataset(dataset_df, validation_size=5000)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(validation_df)}")

It's recommended to train and validate classification tasks on balanced datasests, so that the model and metrics are unbiased to one label. Let's create a balanced train and validation sets.

In [ ]:
balanced_train_df = balance_dataset(train_df, key="routing_label")
balanced_validation_df = balance_dataset(validation_df, key="routing_label")

visualize_label_distribution(balanced_train_df, key="routing_label")

print(f"Train size: {len(balanced_train_df)}")
print(f"Validation size: {len(balanced_validation_df)}")

# Finetune an LLM classifier

Format data in Anyscale finetuning format

In [ ]:
from data_utils import prepare_ft_messages

balanced_train_df["messages"] = prepare_ft_messages(balanced_train_df)
balanced_validation_df["messages"] = prepare_ft_messages(balanced_validation_df)

# for debugging, here's what the messages look like:
display(balanced_train_df["messages"].iloc[0])

TODO: Train llm classifier

#  Batch inference with router model

#  Evaluate router performance

## Baselines
1. Random

In [ ]:
from collections import OrderedDict
router_predictions = OrderedDict()

In [ ]:
import numpy as np
rng = np.random.RandomState(123)
router_predictions["Random"] = rng.uniform(0, 1, len(validation_df))


In [ ]:
from evaluation_metrics import plot_quality_cost_curve

oss_model_scores = validation_df['label'].to_numpy()
closed_model_scores = np.ones(len(validation_df['label'])) * 5.0

plot_quality_cost_curve(oss_model_scores, closed_model_scores, router_predictions)